In [1]:
import os
import pandas as pd

DATA_DIR = '/Users/singhn4/Projects/kids_first/data/Seidman_2015'
DBGAP_DIR = os.path.join(DATA_DIR, 'dbgap')
ALIQUOT_SHIP_DIR = os.path.join(DATA_DIR, 'manifests', 'shipping')

### Extraction - Methods

In [2]:
# Helper functions
def dropna_rows_cols(df_func):
    """
    Decorator to drop rows and cols w all nan values
    """

    def wrapper(*args, **kwargs):
        df = df_func(*args, **kwargs)

        # None or empty df
        try:
            if df.empty:
                return df
        except AttributeError:
            return df

        # Rows
        df.dropna(how="all", inplace=True)
        # Cols
        df.dropna(how="all", axis=1, inplace=True)
        return df

    return wrapper

def reformat_column_names(df_func):
    """
    Decorator to reformat DataFrame column names.

    Replace all column names having whitespace with underscore
    and make lowercase
    """

    def wrapper(*args, **kwargs):
        df = df_func(*args, **kwargs)
        # None or empty df
        try:
            if df.empty:
                return df
        except AttributeError:
            return df
        df.columns = map((lambda x: x.replace(" ", "_").lower()),
                         df.columns)
        return df

    return wrapper

In [3]:
# Study
# dbgap_link = https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs001138.v1.p2
study = {
    'data_access_authority': 'dbGaP',
    'external_id': 'phs001138',
    'version': 'v1.p2',
    'name': 'National Heart, Lung, and Blood Institute (NHLBI) Bench to Bassinet Program: The Gabriella Miller Kids First Pediatric Research Program of the Pediatric Cardiac Genetics Consortium (PCGC)',
    'attribution': 'https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/GetAcknowledgementStatement.cgi?study_id=phs001138.v1.p2'
}    

In [4]:
# Family
@reformat_column_names
@dropna_rows_cols
def read_family_data(filepath=None):
    """
    Read family data for all subjects
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR,
                                '7a_dbGaP_PedigreeDS.txt')
    df = pd.read_csv(filepath,
                     delimiter='\t',
                     dtype={'SUBJID': str})
    # Subset of columns
    df.drop(['SEX'], axis=1, inplace=True)
    
    # Add proband column
    def func(row): return bool(row['MOTHER'] and row['FATHER'])
    df['is_proband'] = df.apply(func, axis=1)
    
    return df

In [5]:
# Phenotype


In [6]:
# Outcome

In [7]:
# Gender
@reformat_column_names
@dropna_rows_cols
def read_gender_data(filepath=None):
    """
    Read gender data for all subjects
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR,
                                '3a_dbGaP_SubjectPhenotypes_GenderDS.txt')
    df = pd.read_csv(filepath,
                     delimiter='\t',
                     dtype={'SUBJID': str})

    return df

In [8]:
# Demographic
@reformat_column_names
@dropna_rows_cols
def read_demographic_data(filepaths=None):
    """
    Read demographic data for all subjects (child, mother, father)
    """
    if not filepaths:
        filenames = ['3a_dbGaP_SubjectPhenotypes_DemographicsDS.txt',
                     '3a_dbGaP_SubjectPhenotypes_MaternalDemographicsDS.txt',
                     '3a_dbGaP_SubjectPhenotypes_PaternalDemographicsDS.txt']

        filepaths = [os.path.join(DBGAP_DIR, filename)
                     for filename in filenames
                     ]

    child_demo_df = pd.read_csv(os.path.join(filepaths[0]),
                                delimiter='\t',
                                dtype={'SUBJID': str})

    mother_demo_df = pd.read_csv(os.path.join(filepaths[1]),
                                 delimiter='\t',
                                 dtype={'SUBJID': str})

    father_demo_df = pd.read_csv(os.path.join(filepaths[2]),
                                 delimiter='\t',
                                 dtype={'SUBJID': str})

    # Combine demographics of all subjects
    subject_demo_df = pd.concat(
        [child_demo_df, mother_demo_df, father_demo_df])

    subject_demo_df.drop_duplicates('SUBJID', inplace=True)
    
    # Subset of columns
    subject_demo_df = subject_demo_df[['RACE', 'ETHNICITY', 'SUBJID']]

    def func(row): return "_".join(['demographic', str(row.name)])
    subject_demo_df['demographic_id'] = subject_demo_df.apply(func, axis=1)


    return subject_demo_df

In [9]:
# Diagnosis
@reformat_column_names
@dropna_rows_cols
def read_diagnosis_data(filepath=None):
    """
    Read diagnoses data for all subjects
    """
    if not filepath:
        filename = '3a_dbGaP_SubjectPhenotypes_PatientDiagnosisDS.txt'
        filepath = os.path.join(DBGAP_DIR, filename)

    diagnosis_df = pd.read_csv(filepath,
                               delimiter='\t',
                               dtype={'SUBJID': str})

    def func(row): return "_".join(['diagnosis', str(row.name)])
    diagnosis_df['diagnosis_id'] = diagnosis_df.apply(func, axis=1)

    return diagnosis_df

In [10]:
# Sample
@reformat_column_names
@dropna_rows_cols
def read_subject_sample_data(filepath=None):
    """
    Read sample metadata for all subjects
    """
    if not filepath:
        filename = '6a_dbGaP_SubjectSampleMappingDS.txt'
        filepath = os.path.join(DBGAP_DIR, filename)

    subject_sample_df = pd.read_csv(filepath,
                                    delimiter='\t',
                                    dtype={'SUBJID': str})
    subject_sample_df.drop_duplicates('SUBJID', inplace=True)

    return subject_sample_df

In [11]:
# Aliquot
@reformat_column_names
@dropna_rows_cols
def read_sample_shipping_manifest_data(*filepaths):
    """
    Read shipping manifest for samples (from PI/sample source center)
    """
    if not filepaths:
        filepaths = [os.path.join(ALIQUOT_SHIP_DIR, filename)

                     for filename in os.listdir(ALIQUOT_SHIP_DIR)
                     ]

    # Combine all manifest files
    dfs = [pd.read_excel(filepath,
                         delimiter='/t',
                         dtype={'*barcode': str},
                         skiprows=[0, 1],
                         header=[6])

           for filepath in filepaths

           if os.path.basename(filepath).startswith("PCGC")

           ]
    df = pd.concat(dfs)

    # Rename columns
    df.columns = map((lambda x: x.lower().lstrip("*")), df.columns)
    
    # Subset of columns
    df = df[['barcode',
             'external_id',
             'sample_collection_site',
             'sample_role',
             'concentration_ng_per_ul',
             'initial_volume_microliters']]

    # Drop rows where id cols are nan
    id_cols = [col for col in df.columns if "id" in col]
    df.dropna(subset=id_cols, inplace=True)

    return df

In [26]:
# Sequencing experiment (from read group metadata)
@reformat_column_names
@dropna_rows_cols
def read_seq_experiment_data(filepath=None):
    if not filepath:
        filepath = os.path.join(DATA_DIR, "seidman_metadata.xlsx")

    df = pd.read_excel(filepath, dtype={"date": str})
    # Rename some columns
    df.rename(columns={"library_name (in original BAM header)":
                       "library_name",
                       "barcode": "rg_barcode"}, inplace=True)
    df["read_length"] = df["read_length"].apply(
        lambda x: int(x.split("x")[0]))
    
    # Create new columns
    df['max_insert_size'] = df['insert_size'].max()
    df['mean_insert_size'] = df['insert_size'].mean()
    df['mean_read_length'] = df['read_length'].mean()
    df['total_reads'] = df['read_length'].count()
    
    # Subset of columns
    df = df[['sample_name',
             'library_name',
             'rg_barcode',
             'run_name',
             'read_length',
             'date',
             'library_strategy',
             'library_source',
             'library_selection',
             'insert_size',
             'instrument',
             'library_layout',
             'max_insert_size',
             'mean_insert_size',
             'mean_read_length',
             'total_reads']]
    
    return df

### Extraction - Execution

In [13]:
# Family 
family_df = read_family_data()
family_df.head()

,famid,subjid,mother,father,is_proband
0,1,210,211,212,True
1,1,211,0,0,False
2,1,212,0,0,False
3,2,272,273,274,True
4,2,273,0,0,False


In [14]:
# Gender
gender_df = read_gender_data()
gender_df.head()

,subjid,sex
0,5787,M
1,16252,F
2,26290,F
3,14665,F
4,2248,M


In [15]:
# Demographic
demographic_df = read_demographic_data()
demographic_df.head()

,race,ethnicity,subjid,demographic_id
0,White,No,3673,demographic_0
1,Black or African American,No,21296,demographic_1
2,White,No,9653,demographic_2
3,White,No,15386,demographic_3
4,White,Yes,26312,demographic_4


In [28]:
# Diagnosis data
diagnosis_df = read_diagnosis_data()
diagnosis_df.head()
# diagnosis_df[diagnosis_df['subjid'] == '279']

,subjid,diagnosis,diagnosis_id
256,279,"Pulmonary stenosis, subvalvar",diagnosis_256
257,279,"Double outlet right ventricle, ventricular def...",diagnosis_257
258,279,Tubular hypoplasia of aorta with coarctation,diagnosis_258
259,279,Single coronary from the left facing cusp,diagnosis_259
260,279,Hypoplastic tricuspid valve,diagnosis_260
261,279,Dysplastic pulmonary valve,diagnosis_261
262,279,Straddling mitral valve,diagnosis_262
263,279,"Ventricular septal defect, malalignment",diagnosis_263
264,279,Left aortic arch,diagnosis_264


In [17]:
# Sample data
subject_sample_df = read_subject_sample_data()
subject_sample_df.head()

,subjid,sampid,samp_source,source_sampid,sample_use
0,22921,CG0019-7254,CORIELL,CG0019-7254,Seq_DNA_WholeGenome
1,16165,CG0012-4474,CORIELL,CG0012-4474,Seq_DNA_WholeGenome
2,9800,CG0009-6098,CORIELL,CG0009-6098,Seq_DNA_WholeGenome
3,15386,CG0011-4770,CORIELL,CG0011-4770,Seq_DNA_WholeGenome
4,570,CG0000-1854,CORIELL,CG0000-1854,Seq_DNA_WholeGenome


In [18]:
# Aliquot/Sample Shipping data
shipping_manifest_df = read_sample_shipping_manifest_data()
shipping_manifest_df.head()

,barcode,external_id,sample_collection_site,sample_role,concentration_ng_per_ul,initial_volume_microliters
0,1125743864,CG0000-6676,Blood,Affected,50.0,66.1
1,1125743887,CG0000-6978,Blood,Parent,50.0,65.7
2,1125743888,CG0000-6989,Blood,Parent,50.0,66.0
3,1125743911,CG0000-6731,Blood,Affected,50.0,65.7
4,1125743912,CG0000-6738,Blood,Parent,50.0,66.2


In [25]:
# Sequencing experiments
seq_exp_df = read_seq_experiment_data()
seq_exp_df.head()

,sample_name,bam_library_name,library_name,rg_barcode,run_name,read_length,date,library_strategy,library_source,library_selection,insert_size,std_dev,instrument,base_caller,quality_scorer,library_layout,max_insert_size,mean_insert_size,mean_read_length,total_reads
0,CG0023-9153,GMKFCS.ST-00000936-1_2AMP,GMKFCS.ST-00000936-1_2AMP,HMNMFCCXX-1,160423_ST-E00212_0248_AHMNMFCCXX_1,150,2016-05-10 00:00:00,WGS,GENOMIC,RANDOM,361,0.1,HiSeq X Ten,Casava V1.8.3,Casava V1.8.3,Paired-End,428,392.684444,150.0,900
1,CG0002-6325,GMKFCS.ST-00000953-1_2AMP,GMKFCS.ST-00000953-1_2AMP,HMNMFCCXX-2,160423_ST-E00212_0248_AHMNMFCCXX_2,150,2016-05-10 00:00:00,WGS,GENOMIC,RANDOM,369,0.1,HiSeq X Ten,Casava V1.8.3,Casava V1.8.3,Paired-End,428,392.684444,150.0,900
2,CG0002-6931,GMKFCS.ST-00000954-1_2AMP,GMKFCS.ST-00000954-1_2AMP,HMNMFCCXX-3,160423_ST-E00212_0248_AHMNMFCCXX_3,150,2016-05-10 00:00:00,WGS,GENOMIC,RANDOM,360,0.1,HiSeq X Ten,Casava V1.8.3,Casava V1.8.3,Paired-End,428,392.684444,150.0,900
3,CG0002-7519,GMKFCS.ST-00003538-1_2AMP,GMKFCS.ST-00003538-1_2AMP,HMNMFCCXX-4,160423_ST-E00212_0248_AHMNMFCCXX_4,150,2016-05-10 00:00:00,WGS,GENOMIC,RANDOM,365,0.1,HiSeq X Ten,Casava V1.8.3,Casava V1.8.3,Paired-End,428,392.684444,150.0,900
4,CG0002-7976,GMKFCS.ST-00003539-1_2AMP,GMKFCS.ST-00003539-1_2AMP,HMNMFCCXX-5,160423_ST-E00212_0248_AHMNMFCCXX_5,150,2016-05-10 00:00:00,WGS,GENOMIC,RANDOM,352,0.1,HiSeq X Ten,Casava V1.8.3,Casava V1.8.3,Paired-End,428,392.684444,150.0,900


### Combine

In [ ]:
# Create participant df
# Merge Gender + Demographics
gender_demo_df = pd.merge(gender_df, demographic_df, on='subjid')
# Add Family
df1 = pd.merge(gender_demo_df, family_df, on='subjid')
df1.head()

In [ ]:
# Merge Diagnosis
df2 = pd.merge(df1, diagnosis_df, on='subjid')
df2.head()

In [ ]:
# Merge Sample
df3 = pd.merge(df2, subject_sample_df, on='subjid')
df3.head()

In [ ]:
# Merge Aliquot
df4 = pd.merge(df3, shipping_manifest_df, left_on='sampid', right_on='external_id')
df4.head()

In [ ]:
# Merge Sequencing Experiment
participant_df = pd.merge(df4, seq_exp_df, left_on='external_id', right_on='sample_name')
participant_df.head()